In [ ]:

# Full archive search

import requests
import os
import json
import pandas as pd
import csv
import datetime
import dateutil.parser
import unicodedata
import time
import copy

os.environ[
    'TOKEN'] = 'AAAAAAAAAAAAAAAAAAAAAH%2BqVwEAAAAA109XqRhzWkEAIaGxKVHemrOhRug%3DokS8leyFFdRepJK9n2le1Xzirhi8Vo9I9VJyivOPhL1XqphHsB'


def auth():
    return os.getenv('TOKEN')


def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers


def create_url(keyword, start_date, end_date, max_results=10):
    search_url = "https://api.twitter.com/2/tweets/search/all"  # Change to the endpoint you want to collect data from

    # change params based on the endpoint you are using
    query_params = {'query': keyword,
                    'start_time': start_date,
                    'end_time': end_date,
                    'max_results': max_results,
                    'expansions': 'author_id',
                    # in_reply_to_user_id,geo.place_id, attachments.poll_ids'
                    'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
                    'user.fields': 'id,name,username,created_at,description,public_metrics',
                    'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                    'next_token': {}}
    return (search_url, query_params)


def connect_to_endpoint(url, headers, params, next_token=None):
    params['next_token'] = next_token  # params object received from create_url function
    response = requests.request("GET", url, headers=headers, params=params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json(), response


# Inputs for the request
bearer_token = auth()
headers = create_headers(bearer_token)
start_time = "2022-03-20T00:00:00.000Z"
end_time = "2022-03-21T00:00:00.000Z"
max_results = 500

# first request to set a dataframe
url = create_url('DIABETES', start_time, end_time, max_results)
json_response, response = connect_to_endpoint(url[0], headers, url[1])
df = pd.DataFrame(json_response['data'])
users = pd.DataFrame(json_response['includes']['users'])

for keyword in ["DIABETES", "T1D", "T2D", "INSULIN"]:
    url = create_url(keyword, start_time, end_time, max_results)
    # request
    json_response, response = connect_to_endpoint(url[0], headers, url[1])
    if keyword != "DIABETES":
        df_temp = pd.DataFrame(json_response['data'])
        users_temp = pd.DataFrame(json_response['includes']['users'])
        df = df.append(df_temp)
        users = users.append(users_temp)
        ## to retrieve all of the results query
    while True:
        try:
            if json_response['meta']['next_token']:
                json_response, response = connect_to_endpoint(url[0], headers, url[1],
                                                              next_token=json_response['meta']['next_token'])
                df_temp = pd.DataFrame(json_response['data'])
                users_temp = pd.DataFrame(json_response['includes']['users'])
                df = df.append(df_temp)
                users = users.append(users_temp)
        except:
            break

In [ ]:

# creating dataframe for tagging

Left_join = pd.merge(df,
                     users,
                     left_on='author_id',
                     right_on='id',
                     how='left')

author_id_list = []
number_of_tweets = []
tweets = []
description = []

new_users_taggign = pd.DataFrame()

for authorId in Left_join['author_id'].unique(): #loop for all author_id
    df_temp = Left_join[Left_join['author_id']==authorId]
    all_user_tweets = len(df_temp)

    user_tweets = []
    for idx, row in df_temp.iterrows():

        user_tweets.append(row['text'])

    tweets.append(user_tweets)
    description.append(row['description'])
    author_id_list.append(authorId)
    number_of_tweets.append(all_user_tweets)


new_users_taggign['author_id_list'] = author_id_list
new_users_taggign['number_of_tweets'] = number_of_tweets
new_users_taggign['tweets'] = tweets
new_users_taggign['description'] = description